### Importing necessary libraries

In [1]:
import numpy as np
import pandas as pd
from pyclustering.cluster.clique import clique
from pyclustering.cluster.clique import clique_visualizer

### Implementing CLIQUE algorithm

In [ ]:
intervals = ?
# Density threshold
threshold = ?
number_of_features = ?
clique_instance = clique(np.array(df.iloc[:,:number_of_features]), intervals, threshold)
# clique_instance = clique(data_M, intervals, threshold)
# Start the clustering process and get the results
clique_instance.process()
clique_cluster = clique_instance.get_clusters()  # allocated clusters

# Points considered as outliers (noise points)
noise = clique_instance.get_noise()
# CLIQUE formed grid unit
cells = clique_instance.get_cells() 
num_of_clusters = len(clique_cluster)
print("Amount of clusters:", num_of_clusters)
print(clique_cluster)

#determining the label of each data point based on CLIQUE cluster
num_of_clusters = len(clique_cluster)
data_cluster = {}
for i in range(num_of_clusters):
    for item in clique_cluster[i]:
        data_cluster[item] = i
        
#identifying cells which is related to a cluster        
cell_cluster = {}
for j in range(len(cells)):
    try:
        cell_cluster[cells[j]] = data_cluster.get(cells[j].points[0])
    except:
        cell_cluster[cells[j]] = -1
        
        
for i in range(len(df)):
    if i not in data_cluster.keys():
        data_cluster[i] = -1
        
labelsss = []
for item in sorted(data_cluster.items(), key=lambda x: x[0]):
    labelsss.append(item[1])
    
df["Cluster"] = labelsss
label_list = set(labelsss) - {-1}
label_list = sorted(label_list)
max_cell = np.zeros((num_of_clusters,number_of_features))
min_cell = np.zeros((num_of_clusters,number_of_features))

for item in label_list:
    temp = df[df["Cluster"] == item].iloc[:,:number_of_features]
    min_cell[item,:] = np.array(np.min(temp, axis=0))
    max_cell[item,:] = np.array(np.max(temp, axis=0))

### Defining major fucntions for RDCA

In [ ]:
def get_back_cell(item):
    for elm in cells:
        if str(elm) == str(item):
            return elm.spatial_location.get_corners()
        
#==========================================================================================================================    

def cell_location(index_of_cluster):
    filterd_dict = {}
    filterd_dict = {k:v for (k,v) in cell_cluster.items() if v == index_of_cluster}
    return (filterd_dict, len(filterd_dict))

#==========================================================================================================================    

def RDA_initial_population(index_of_cluster,kind):
    
    global num_of_population
    num_of_population = 40
    initial_population = []
    
    if kind == "my_method":
        initial_population = np.random.uniform(min_cell[index_of_cluster,:], max_cell[index_of_cluster,:], size=(num_of_population, number_of_features)).tolist()
        return initial_population
    
    elif kind == "normal":
        temp = []
        for iterr in range(num_of_population):
            for i in range(number_of_features):
                temp.append(np.random.uniform(min_borders[i], max_borders[i]))
            initial_population.append(temp)
            temp = []
        return initial_population 
    
#==========================================================================================================================    
                   
def compatness_measure(point):
    global within_cluster_result
    within_cluster_result = {}

    for item in point:
        matrix = np.zeros((len(df), num_of_clusters))
        for i in range(len(point[item])):
            matrix[:,i] = np.linalg.norm(np.broadcast_to(point[item][i], (len(df),number_of_features)) - np.array(df.iloc[:,:number_of_features]), axis=1).tolist()
    
        matrix = np.concatenate((matrix.min(axis=1).reshape(len(df),1),np.argmin(matrix,axis=1).reshape(len(df),1)), axis=1)
        within_cluster_result[item] = np.mean(npi.group_by(matrix[:, 1]).mean(matrix[:,0])[1])  
        
    return within_cluster_result

#==========================================================================================================================

def get_labels(point):
    global labels
    labels = {}
    
    for item in point:
        matrix2 = np.ones((len(df), num_of_clusters))*np.inf
        for i in range(len(point[item])):
            matrix2[:,i] = np.linalg.norm(np.broadcast_to(point[item][i], (len(df),number_of_features)) - np.array(df.iloc[:,:number_of_features]), axis=1).tolist()
        labels[item] = np.argmin(matrix2,axis=1)
    return labels
    
#==========================================================================================================================    

def seperation_measure(point):
    global between_cluster_distance
    between_cluster_distance = {}
    tmp = 0
    for item in point:
        if len(point[item]) != 1:
            for i in range(len(point[item])):
                for j in range(i+1,len(point[item])):
                    tmp += (np.linalg.norm(np.array(point[item][i])-np.array(point[item][j])))
            between_cluster_distance[item] = tmp/(comb(len(point[item]), 2))
            tmp = 0
        else:
            between_cluster_distance[item] = np.linalg.norm(np.array(point[item][0]))
    return between_cluster_distance

#==========================================================================================================================    

def combined_measure(point):
    global w1,w2
    compactness_result = compatness_measure(point)
    separation_result = seperation_measure(point)
    
    combined_result = {}
    for i in point:
        flag = 0
        for k in range(len(point[i])):
            for j in range(len(point[i][k])):
                if point[i][k][j] > max_borders[j] or point[i][k][j] < min_borders[j]:
                    flag = 1
                    break
        if flag == 1:
            combined_result[i] = 10**10
        else:
            combined_result[i] = (w1*compactness_result[i]) - (w2*separation_result[i])
    return combined_result

#==========================================================================================================================

turi_random_value = np.random.normal(2,1)
def turi_index_measure(point):
    global min_value_turi,turi_measure_result
    turi_measure_result = {}
    intra = compatness_measure(point)
    c = 1
    turi_index_value = []
    min_value_turi = np.inf
    for item in point:
        if len(point[item]) != 1:
            for i in range(len(point[item])):
                for j in range(i+1,len(point[item])):
                    min_value_turi = min((np.linalg.norm(np.array(point[item][i])-np.array(point[item][j]))), min_value_turi)
                    turi_measure_result[item] = intra[item]/min_value_turi
        else:
            min_value_turi = np.linalg.norm(np.array(point[item][0]))
            turi_measure_result[item] = intra[item]/min_value_turi
        
    return turi_measure_result

#==========================================================================================================================    

def S_Dbw_measure(point):
    s_dbw_measure_result = {}
    label = get_labels(point)
    
    for item in point:
        if len(set(get_labels({0: point[item]})[0])) >= 2:
            s_dbw_measure_result[item] = S_Dbw(np.array(df.iloc[:, :number_of_features]),label[item] , centers_id=None, method='Tong', alg_noise='bind',
        centr='mean', nearest_centr=True, metric='euclidean')
        else:
            s_dbw_measure_result[item] = 10**5
    return s_dbw_measure_result

#==========================================================================================================================

def dunn_index_measure(point):
    point_index = list(point.keys())[0]
    label = get_labels(point)[point_index]
    temp = list(set(label))
     
    data_points = [[] for number in range(len(temp))]
    for i in range(len(label)):
        data_points[label[i]].append(i)

    min_value_dunn = np.inf
    max_value_dunn = -np.inf
    for i in range(len(data_points)):
        for j in range(len(data_points[i])):
            for m in range(j+1, len(data_points[i])):
                max_value_dunn = max(max_value_dunn,(np.linalg.norm(np.array(df.iloc[data_points[i][j],:number_of_features]) - np.array(df.iloc[data_points[i][m], :number_of_features]))))



    for i in range(len(data_points)):
        for j in range(i+1, len(data_points)):
            for m in data_points[i]:
                for n in data_points[j]:
                    min_value_dunn = min(min_value_dunn,(np.linalg.norm(np.array(df.iloc[m,:number_of_features]) - np.array(df.iloc[n, :number_of_features]))))

    min_value_dunn = min_value_dunn / max_value_dunn
    return min_value_dunn

#==========================================================================================================================

def db_measure(point):
    global db_measure_result
    db_measure_result = {}
    for item in point:
        if len(set(get_labels({0: point[item]})[0])) >= 2:
            db_measure_result[item] = davies_bouldin_score(df.iloc[:, :number_of_features], get_labels({0: point[item]})[0])
        else:
            db_measure_result[item] = 10**5
    return db_measure_result

#==========================================================================================================================

def silhouette_measure(point):
    global silhouette_measure_result
    silhouette_measure_result = {}
    for item in point:
        if len(set(get_labels({0: point[item]})[0])) >= 2:
            silhouette_measure_result[item] = silhouette_score(df.iloc[:, :number_of_features], get_labels({0: point[item]})[0])
        else:
            silhouette_measure_result[item] = -10**5
    return silhouette_measure_result

#==========================================================================================================================

def ARI_measure(point):
    ari_measure_result = {}
    label = get_labels(point)
    
    for item in point:
        if len(set(get_labels({0: point[item]})[0])) >= 2:
            ari_measure_result[item] = adjusted_rand_score(df['class'], label[0])
        else:
            ari_measure_result[item] = 10**5
    return ari_measure_result

#==========================================================================================================================

def nmi_measure(point):
    nmi_measure_result = {}
    label = get_labels(point)
    
    for item in point:
        if len(set(get_labels({0: point[item]})[0])) >= 2:
            nmi_measure_result[item] = normalized_mutual_info_score(df['class'], label[0])
        else:
            nmi_measure_result[item] = 10**5
    return nmi_measure_result

#==========================================================================================================================

def custom_function(point):
    global custom_function_result
    custom_function_result = {}
    
    for item in point:
        custom_function_result[item] = (np.sum(np.array(point[item])**2))
    return custom_function_result

#==========================================================================================================================

def fitness_function (point, measure):
    if measure == 'compactness':
        return compatness_measure(point)
    elif measure == 'separation':
        return seperation_measure(point)
    elif measure == 'combined':
        return combined_measure(point)
    elif measure == 'turi':
        return turi_index_measure(point)
    elif measure == 'db':
        return db_measure(point)
    elif measure == 'nmi':
        return nmi_measure(point)
    elif measure == 'silhouette':
        return silhouette_measure(point)
    elif measure == 'custom':
        return custom_function(point)
#==========================================================================================================================

def generate_candidates():
    global candidates, random_candidates
    candidates = {}
    can_index = 0
    number_of_samples = 150
    random_candidates = np.random.randint(0,num_of_population,(int(number_of_samples*1.25),num_of_clusters))
    random_candidates = np.unique(random_candidates, axis=0)
    for item in random_candidates:
        can_element = []
        c = 0
        for item2 in item:
            can_element.append(population[c][item2])
            c += 1
        candidates[can_index] = can_element
        can_index += 1
    return candidates

#==========================================================================================================================    

def RDA_select_Nmale():
    global Nmales, Nhinds
    Nmales = {}
    Nhinds = {}
    alpha = 0.15
    num_of_Nmale = np.round(alpha*(len(random_candidates)), 0)
#     num_of_Nmale = 15
    i = 0
    if optimize_method == 'min':
        sorted_fitness_measure = sorted(fitness_result.items(), key= lambda x: x[1])
    elif optimize_method == 'max':
        sorted_fitness_measure = sorted(fitness_result.items(), key= lambda x: x[1], reverse=True)
    for item in sorted_fitness_measure :
        if i < num_of_Nmale:
            Nmales[item[0]] = candidates[item[0]]
            i += 1
        else:
            Nhinds[item[0]] = candidates[item[0]]

#==========================================================================================================================    

min_borders = []
max_borders = []
for num in range(number_of_features):
    min_borders.append(min(df.iloc[:, num]))
    max_borders.append(max(df.iloc[:, num]))
    
def RDA_find_ub_lb():
    global UB, LB
    UB = np.broadcast_to(1*max_borders, (num_of_clusters, number_of_features))
    LB = np.broadcast_to(1*min_borders, (num_of_clusters, number_of_features))
    
#==========================================================================================================================

def RDA_roaring(kind):
    global new
    for item in Nmales:
        new = []
        a3 = np.random.uniform()
        a2 = np.random.uniform()
        a1 = np.random.uniform()
        if a3 >= 0.5:
            new = (Nmales[item] + a1*(((UB - LB)*a2) +LB)).tolist()
        else:
            new = (Nmales[item] - a1*(((UB - LB)*a2) +LB)).tolist()
        
        
        if kind == 'my_method':
            find_mean({6666: Nmales[item]})
            a = fitness_function({0: new}, measure)[0]
            b = fitness_result[item]
            c = fitness_function({0: mean[6666]}, measure)[0]

            if optimize_method == 'min':
                temp_result = np.argmin([a, b, c])
            elif optimize_method == 'max':
                temp_result = np.argmax([a, b, c])
                
        elif kind == 'normal':
            a = fitness_function({0: new}, measure)[0]
            b = fitness_result[item]

            if optimize_method == 'min':
                temp_result = np.argmin([a, b])
            elif optimize_method == 'max':
                temp_result = np.argmax([a, b])
                
                
        if temp_result == 0:
            Nmales[item] = new
            candidates[item] = new
            fitness_result[item] = a
      
        elif temp_result == 2:
            Nmales[item] = mean[6666]
            candidates[item] = mean[6666]
            fitness_result[item] = c
#==========================================================================================================================

def RDA_select_commanders():
    global commanders, stags,after_roaring_result
    commanders = {}
    stags = {}
    gamma = 0.7
    num_of_commanders = np.round(gamma*len(Nmales), 0)
    i = 0
    after_roaring_result = fitness_function(Nmales, measure)
    
    if optimize_method == 'min':
        sorted_Nmales = sorted(after_roaring_result.items(), key= lambda x: x[1])
    elif optimize_method == 'max':
        sorted_Nmales = sorted(after_roaring_result.items(), key= lambda x: x[1], reverse=True)
        
    for item in sorted_Nmales:
        if i < num_of_commanders:
            commanders[item[0]] = Nmales[item[0]]
            i += 1
        else:
            stags[item[0]] = Nmales[item[0]]     

#==========================================================================================================================

def find_mean(point):
    #point should be in dictionary format
    global mean
    mean = {}
    point_index = list(point.keys())[0]
    label = get_labels(point)[point_index]
    mean_matrix = np.concatenate((np.array(df.iloc[:, :number_of_features]),label.reshape(len(df),-1)),axis=1)
    mean[6666] = npi.group_by(mean_matrix[:, -1]).mean(mean_matrix[:,:-1])[1].tolist()
    if len(mean[6666]) != num_of_clusters:
        label_list = set(label)
        temp_point = point[6666]
        counter = 0
        for label in label_list:
            temp_point[label] = mean[6666][counter]
            counter += 1
        mean[6666] = temp_point
#==========================================================================================================================

def RDA_fight(comm,kind):
    for item in comm:
        b1 = np.random.uniform()
        b2 = np.random.uniform()
        
        stag_index = list(stags.keys())[np.random.randint(len(stags))]
        mianeh = np.array(commanders[item]) + np.array(stags[stag_index])
        new1 = ((mianeh/2) + b1 * (((UB - LB)*b2) + LB)).tolist()
        new2 = ((mianeh/2) - b1 *(((UB - LB)*b2) + LB)).tolist()
        
        if kind == "my_method":
            find_mean({6666: comm[item]})
            a = fitness_function({0: new1},measure)[0]
            b = fitness_function({0: new2},measure)[0]
            c = fitness_result[item]
            d = fitness_result[stag_index]
            e = fitness_function({0: mean[6666]}, measure)[0]
            if optimize_method == 'min':
                temp_result1 = np.argmin([a, b, c, d, e])
            elif optimize_method == 'max':
                temp_result1 = np.argmax([a, b, c, d, e])
                
        elif kind == "normal":
            a = fitness_function({0: new1},measure)[0]
            b = fitness_function({0: new2},measure)[0]
            c = fitness_result[item]
            d = fitness_result[stag_index]
            if optimize_method == 'min':
                temp_result1 = np.argmin([a, b, c, d])
            elif optimize_method == 'max':
                temp_result1 = np.argmax([a, b, c, d])

        if temp_result1 == 0:
            commanders[item] = new1
            Nmales[item] = new1
            candidates[item] = new1
            fitness_result[item] = a
        elif temp_result1 == 1:
            commanders[item] = new2
            Nmales[item] = new2
            candidates[item] = new2
            fitness_result[item] = b
        elif temp_result1 == 3:
            tt = commanders[item] 
            commanders[item] = stags[stag_index]
            stags[stag_index] = tt
            comm_re = fitness_result[item]
            stag_re = fitness_result[stag_index]
            fitness_result[item] = stag_re
            fitness_result[stag_index] = comm_re
            
#==========================================================================================================================
        
def RDA_form_harems(comm):
    global Nharem, num_of_harem_members
    Nharem = {}
    num_of_harem_members = {}

    
    sum_of_vi = 0
    
    if optimize_method == 'min':
        maximum_vi = -np.inf
        for item in comm:
            maximum_vi = max(fitness_result[item], maximum_vi)
            
    elif optimize_method == 'max':
        maximum_vi = np.inf
        for item in comm:
            maximum_vi = min(fitness_result[item], maximum_vi)
            
    for item in comm:
        sum_of_vi += (fitness_result[item] - maximum_vi)
        
    Nhinds_index = list(Nhinds.keys())    
    for item in comm:
        temp = []
        if sum_of_vi == 0:
            num_of_harem_members[item] = int(len(Nhinds)/len(comm))
        else:
            num_of_harem_members[item] = int(np.round(np.abs(((fitness_result[item] - maximum_vi)/sum_of_vi)*len(Nhinds)), 0))
        for i in range(min(num_of_harem_members[item], len(Nhinds_index))):
            random_index = np.random.randint(len(Nhinds_index))
            temp.append(Nhinds[Nhinds_index[random_index]])
            Nhinds_index.remove(Nhinds_index[random_index])
        Nharem[item] = temp
        
#==========================================================================================================================        
        
def RDA_comm_mate_with_its_harem(nharem):
    
    global offspring
    alpha = 0.9
    offspring = {}
    offspring_index = len(candidates)
    
    for item in nharem:
        num_of_mate = alpha * len(Nharem[item])
        
        if Nharem[item] != []:
            Nharem_index = list(range(len(Nharem[item])))
            
            for i in range(int(num_of_mate)):
                Nharem_random_index = random.choice(Nharem_index)
                c = np.random.uniform()
                
                mianeh = (np.array(commanders[item]) + np.array(Nharem[item][Nharem_random_index])/2)
                temp = (mianeh + ((UB - LB) * c)).tolist()
                
                Nharem_index.remove(Nharem_random_index)
                offspring[offspring_index] = temp
                offspring_index += 1

#==========================================================================================================================        
        
def RDA_comm_mate_with_other_harem(comm):
    global selected_harem, Nharem_index
    beta = 0.4
    offspring_index = len(offspring) + len(candidates)
    for item in comm:
        harem_list = list(Nharem.keys())
        harem_list.remove(item)
        count = 0
        while True:

            selected_harem = random.choice(harem_list)
            count += 1
            if Nharem[selected_harem] != [] or count == 10:
                break

        num_of_mate = int(beta * len(Nharem[selected_harem]))

        Nharem_index = list(range(len(Nharem[selected_harem])))
        for i in range(num_of_mate):
            Nharem_random_index = random.choice(Nharem_index)
            c = np.random.uniform()
            mianeh = (np.array(commanders[item]) + np.array(Nharem[selected_harem][Nharem_random_index])/2)
            temp = (mianeh + ((UB -LB) * c)).tolist()

            Nharem_index.remove(Nharem_random_index)
            offspring[offspring_index] = temp
            offspring_index += 1
                
# #==========================================================================================================================

    
def RDA_stag_mate_with_nearest_hind(stagss):
    offspring_index = len(offspring) + len(candidates)
    global stag_hind_mate
    stag_hind_mate = {}
    for item in stagss:
        temp = []
        for hind in Nhinds:
            temp.append(np.linalg.norm(np.array(stags[item]) - np.array(Nhinds[hind])))
        stag_hind_mate[item] = list(Nhinds.keys())[temp.index(min(temp))]
    
    for item in stag_hind_mate:
        c = np.random.uniform()
        mianeh = (np.array(stags[item]) + np.array(Nhinds[stag_hind_mate[item]])/2)
        temp2 = (mianeh + ((UB - LB) * c)).tolist()

        offspring[offspring_index] = temp2
        offspring_index += 1

#==========================================================================================================================

def RDA_next_generation():
    global all_result, next_generation_index, next_generation_Nhinds, next_generation_Nmales, new_candidates, Nmales, candidates, Nhinds, fitness_result
    all_result = {}
    next_generation_index = 0
    lists = [fitness_result, fitness_offs_result]
    for item in lists:
        for item2 in item:
            all_result[item2] = item[item2]
            
    if optimize_method == 'min':
        sorted_all_result = sorted(all_result.items(), key= lambda x: x[1])
    elif optimize_method == 'max':
        sorted_all_result = sorted(all_result.items(), key= lambda x: x[1], reverse=True)
        
    new_candidates = {}
    next_generation_Nmales = {}
    temp_hinds = {}
    i = 0

    for elemnt in sorted_all_result:
        if i < len(Nmales):
            try:
                next_generation_Nmales[next_generation_index] = candidates[elemnt[0]]
            except:
                next_generation_Nmales[next_generation_index] = offspring[elemnt[0]]
            next_generation_index += 1   
            i += 1
        else:
            temp_hinds[elemnt[0]] = all_result[elemnt[0]]
    
    if optimize_method == 'min':
        max_value = max(temp_hinds.values())
    elif optimize_method == 'max':
        max_value = min(temp_hinds.values())
        
    new_values = {}
    p_values = []
    next_generation_Nhinds = {}
    
    if optimize_method == 'min':
        if max_value >= 0:
            for item in temp_hinds:
                new_values[item] = max_value + (max_value - temp_hinds[item])

        else:
            for item in temp_hinds:
                new_values[item] = -temp_hinds[item]
                
    elif optimize_method == 'max':
        if max_value <= 0:
            for item in temp_hinds:
                new_values[item] = max_value + (max_value - temp_hinds[item])

        else:
            for item in temp_hinds:
                new_values[item] = temp_hinds[item]
                
    sum_values = sum(new_values.values())
    for item in new_values:
        p_values.append(new_values[item]/sum_values)

    for i in range(len(Nhinds)):
        index = np.random.choice(list(temp_hinds.keys()), p=p_values)
        try:
            next_generation_Nhinds[next_generation_index] = candidates[index]
        except:
            next_generation_Nhinds[next_generation_index] = offspring[index]
        next_generation_index += 1

    lists2 = [next_generation_Nmales, next_generation_Nhinds]
    new_candidates = {}
    fitness_result = {}
    for item in lists2:
        for item2 in item:
            new_candidates[item2] = item[item2]
            fitness_result[item2] = fitness_function({0: item[item2]}, measure)[0]


        

### RDCA

In [ ]:
number_of_features = ?
num_of_clusters = ?

new_rda_compactness = []
new_rda_separation = []
new_rda_combined = []
new_rda_turi = []
new_rda_dunn = []
new_rda_db = []
new_rda_nmi = []
new_rda_best = []
new_rda_time = []
pop = []
w1,w2 = 0.95,0.05
measure = 'combined'
optimize_method = 'min'
for iterr in np.arange(30):
    start = time.time()
    population = {}
    for i in range(num_of_clusters):
        population[i] = RDA_initial_population(i, 'my_method')

    candidates = generate_candidates()
    fitness_result = fitness_function(candidates,measure)
    RDA_select_Nmale()
    RDA_find_ub_lb()
    temp_result = []
    temp_new_rda_compactness = []
    temp_new_rda_separation = []
    temp_new_rda_combined = []
    temp_new_rda_turi = []
    temp_new_rda_dunn = []
    temp_new_rda_db = []
    temp_new_rda_nmi = []
    temp_new_rda_best = []
    temp_pop = []
    for iteration in range(80):
        RDA_roaring('my_method')
        RDA_select_commanders()
        RDA_fight(commanders,"my_method")
        RDA_form_harems(commanders)
        RDA_comm_mate_with_its_harem(Nharem)
        RDA_comm_mate_with_other_harem(commanders)
        RDA_stag_mate_with_nearest_hind(stags)
        fitness_offs_result = fitness_function(offspring, measure)
        RDA_next_generation()
        Nmales = next_generation_Nmales
        Nhinds = next_generation_Nhinds
        candidates = new_candidates
        alll = sorted(fitness_result.items(), key=lambda x:x[1])
        commander_index = alll[0][0]
        temp_new_rda_compactness.append(fitness_function({0: candidates[commander_index]},'compactness')[0])
        temp_new_rda_separation.append(fitness_function({0: candidates[commander_index]},'separation')[0])
        temp_new_rda_combined.append(fitness_function({0: candidates[commander_index]},'combined')[0])
        temp_new_rda_turi.append(fitness_function({0: candidates[commander_index]},'turi')[0])
        temp_new_rda_db.append(fitness_function({0: candidates[commander_index]},'db')[0])
        temp_new_rda_nmi.append(fitness_function({0: candidates[commander_index]},'nmi')[0])
        temp_new_rda_best.append(candidates[commander_index])
        temp_pop.append(candidates)
        
    end = time.time()
    new_rda_time.append(end - start)
    new_rda_compactness.append(temp_new_rda_compactness)
    new_rda_separation.append(temp_new_rda_separation)
    new_rda_combined.append(temp_new_rda_combined)
    new_rda_turi.append(temp_new_rda_turi)
    new_rda_db.append(temp_new_rda_db)
    new_rda_nmi.append(temp_new_rda_nmi)
    new_rda_best.append(temp_new_rda_best)
    pop.append(temp_pop)
    print("We completed the %ith iteration" %(iterr+1))